# **Modelling and Evaluation - Brain Tumor Detection**

## Objectives



## Inputs



## Outputs




## Comments/Conclusions

---

## Import Packages

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

---

## Change Working Directory

In [ ]:
cwd= os.getcwd()

In [ ]:
os.chdir('/workspace/brain-tumor-detection')
print("You set a new current directory")

In [ ]:
work_dir = os.getcwd()
work_dir

---